Market Basket Analytics

soporte_A=P(A) / sporte_B_B=P(A U B)
confianza = P(B/A) = P(A U B)/P(A) - probabilidad de que se compre el producto B una vez comprado B (si existe relacion entre ambos).
lift= P(A U B)/ P(A)*P(B) -  cuantas veces esta relación es superior al mero azar - cuantifica la fuerza de la relación

In [3]:
import numpy as np
import pandas as pd
import sqlite3

In [9]:
df=pd.read_csv('C:/Users/david/Desktop/Curso ML/Pandas/tickets.csv')

In [11]:
df.head()

,id_pedido,id_cliente,fecha,hora,id_departamento,id_seccion,id_producto,nombre_producto,precio_unitario,cantidad,precio_total
0,1,112108,2023-02-02 00:00:00,10,4,83,49683,Pepino Kirby,0.99,4,3.96
1,1,112108,2023-02-02 00:00:00,10,4,24,13176,Bolsa de Bananas Orgánicas,2.45,4,9.80
2,1,112108,2023-02-02 00:00:00,10,4,24,47209,Aguacate Hass Orgánico,1.79,2,3.58
3,2,202279,2023-02-03 00:00:00,9,4,83,28985,Col Rizada Orgánica de Michigan,2.55,2,5.10
4,2,202279,2023-02-03 00:00:00,9,4,83,17794,Zanahorias,0.95,5,4.75


In [15]:
df.describe()

,id_pedido,id_cliente,hora,id_departamento,id_seccion,id_producto,precio_unitario,cantidad,precio_total
count,4.975718e+06,4.975718e+06,4.975718e+06,4.975718e+06,4.975718e+06,4.975718e+06,4.975718e+06,4.975718e+06,4.975718e+06
mean,1.710496e+06,1.028372e+05,1.341417e+01,4.992144e+00,5.809954e+01,2.862542e+04,1.596929e+00,5.015206e+00,8.009874e+00
std,9.873063e+05,5.950903e+04,4.259586e+00,3.382969e+00,3.909031e+01,1.292617e+04,6.695222e-01,1.983126e+00,4.803981e+00
min,1.000000e+00,1.000000e+00,0.000000e+00,4.000000e+00,1.600000e+01,4.605000e+03,6.200000e-01,1.000000e+00,6.200000e-01
25%,8.552620e+05,5.111900e+04,1.000000e+01,4.000000e+00,2.400000e+01,2.113700e+04,9.900000e-01,4.000000e+00,4.350000e+00
50%,1.710466e+06,1.024700e+05,1.300000e+01,4.000000e+00,2.400000e+01,2.620900e+04,1.490000e+00,5.000000e+00,7.140000e+00
75%,2.565128e+06,1.543180e+05,1.600000e+01,4.000000e+00,8.300000e+01,4.195000e+04,2.250000e+00,6.000000e+00,1.068000e+01
max,3.421083e+06,2.062090e+05,2.300000e+01,2.000000e+01,1.230000e+02,4.968300e+04,2.850000e+00,1.300000e+01,3.705000e+01


In [25]:
df.info() # debemos convertir fecha a datetime

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4975718 entries, 0 to 4975717
Data columns (total 11 columns):
 #   Column           Dtype         
---  ------           -----         
 0   id_pedido        int64         
 1   id_cliente       int64         
 2   fecha            datetime64[ns]
 3   hora             int64         
 4   id_departamento  int64         
 5   id_seccion       int64         
 6   id_producto      int64         
 7   nombre_producto  object        
 8   precio_unitario  float64       
 9   cantidad         int64         
 10  precio_total     float64       
dtypes: datetime64[ns](1), float64(2), int64(7), object(1)
memory usage: 417.6+ MB


In [21]:
df.isnull().sum()

id_pedido          0
id_cliente         0
fecha              0
hora               0
id_departamento    0
id_seccion         0
id_producto        0
nombre_producto    0
precio_unitario    0
cantidad           0
precio_total       0
dtype: int64

In [23]:
# convertimos fechas a datetime
df['fecha']=pd.to_datetime(df['fecha'])

In [29]:
# nos quedamos solo con estas columnas
df_cesta=df[['id_pedido','nombre_producto']]
df_cesta

,id_pedido,nombre_producto
0,1,Pepino Kirby
1,1,Bolsa de Bananas Orgánicas
2,1,Aguacate Hass Orgánico
3,2,Col Rizada Orgánica de Michigan
4,2,Zanahorias
...,...,...
4975713,3421080,Leche Entera Orgánica
4975714,3421080,Racimo de Tomates Orgánicos
4975715,3421080,Cilantro Orgánico
4975716,3421082,Fresas


In [63]:
df_agrupado=df_cesta.groupby('id_pedido')['nombre_producto'].apply(lambda x: ','.join(x))

In [67]:
# agrupamos por pedido y regustramos los productos pedidos en cada pedido. luego lo pasamos adf con reset_index()
df_agrupado=df_agrupado.reset_index()
df_agrupado

,id_pedido,nombre_producto
0,1,"Pepino Kirby,Bolsa de Bananas Orgánicas,Aguaca..."
1,2,"Col Rizada Orgánica de Michigan,Zanahorias"
2,3,Espinacas Baby Orgánicas
3,5,"Bolsa de Bananas Orgánicas,Frambuesas Orgánica..."
4,10,"Banana,Cilantro Orgánico,Aguacate Orgánico,Ceb..."
...,...,...
2060183,3421077,"Frambuesas Orgánicas,Calabacín Orgánico,Leche ..."
2060184,3421078,"Manzanas Gala Orgánicas,Banana"
2060185,3421080,"Leche Entera Orgánica,Racimo de Tomates Orgáni..."
2060186,3421082,Fresas


In [69]:
# aplicamos get_dummies para difercniar cada producto y convertirlo en dato numérico
dummies = df_agrupado['nombre_producto'].str.get_dummies(sep=',')
df_transacciones = pd.concat([df_agrupado[['id_pedido']], dummies], axis=1) # concatenamos df[['id_pedido']] con el df dummies

In [71]:
df_transacciones

,id_pedido,Agua con Gas de Pomelo,Aguacate Hass Orgánico,Aguacate Orgánico,Ajo Orgánico,Apio Orgánico en Ramillete Pequeño,Arándanos Orgánicos,Banana,Bolsa de Bananas Orgánicas,Calabacín Orgánico,...,Manzana Honeycrisp Orgánica,Manzanas Gala Orgánicas,Pepino Kirby,Pepino Orgánico,Racimo de Tomates Orgánicos,Rúcula Baby Orgánica,Tomates Cherry Orgánicos,Uvas Rojas sin Semillas,Zanahorias,Zanahorias Baby Orgánicas
0,1,0,1,0,0,0,0,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,5,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,10,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2060183,3421077,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2060184,3421078,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2060185,3421080,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2060186,3421082,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [83]:
# calculamos cada soporte para cada producto
soporte = df_transacciones.drop(columns=['id_pedido']).mean() * 100
soporte

Agua con Gas de Pomelo                 3.839504
Aguacate Hass Orgánico                10.703004
Aguacate Orgánico                      8.929331
Ajo Orgánico                           5.521778
Apio Orgánico en Ramillete Pequeño     3.427891
Arándanos Orgánicos                    5.086138
Banana                                23.809381
Bolsa de Bananas Orgánicas            19.129710
Calabacín Orgánico                     5.302429
Cebolla Amarilla Orgánica              5.706324
Cebolla Roja Orgánica                  3.429687
Cebollas Amarillas                     3.728592
Cilantro Orgánico                      3.538900
Col Rizada Orgánica de Michigan        3.396632
Coliflor Fresca                        3.597876
Crema                                  3.470606
Crema Orgánica                         3.827855
Espinacas Baby Orgánicas              12.198935
Espárragos                             3.446724
Frambuesas Orgánicas                   6.906797
Fresas                                 7

In [85]:
soporte.sort_values(ascending=False)

Banana                                23.809381
Bolsa de Bananas Orgánicas            19.129710
Fresas Orgánicas                      13.349170
Espinacas Baby Orgánicas              12.198935
Aguacate Hass Orgánico                10.703004
Aguacate Orgánico                      8.929331
Limón Grande                           7.791279
Fresas                                 7.241960
Limones                                7.107944
Leche Entera Orgánica                  6.918301
Frambuesas Orgánicas                   6.906797
Cebolla Amarilla Orgánica              5.706324
Ajo Orgánico                           5.521778
Calabacín Orgánico                     5.302429
Arándanos Orgánicos                    5.086138
Pepino Kirby                           4.840044
Manzana Fuji Orgánica                  4.502841
Limón Orgánico                         4.422266
Tomates Cherry Orgánicos               4.266795
Manzana Honeycrisp Orgánica            4.231216
Uvas Rojas sin Semillas                4

In [113]:
def confianza(antecedente, consecuente):
    conjunto_ac = df_transacciones.loc[
        (df_transacciones[antecedente] == 1) & (df_transacciones[consecuente] == 1)
    ]
    return len(conjunto_ac) / df_transacciones[antecedente].sum()

def lift(antecedente, consecuente):
    soporte_a = df_transacciones[antecedente].mean()
    soporte_c = df_transacciones[consecuente].mean()
    conteo_ac = len(df_transacciones.loc[
        (df_transacciones[antecedente] == 1) & (df_transacciones[consecuente] == 1)
    ])
    soporte_ac = conteo_ac / len(df_transacciones)
    return soporte_ac / (soporte_a * soporte_c)

from itertools import combinations

umbral_confianza = 0.05
asociaciones = []

for antecedente, consecuente in combinations(df_transacciones.columns, 2):
    soporte_a = df_transacciones[antecedente].mean()
    conf = confianza(antecedente, consecuente)
    if conf > umbral_confianza:
        asociaciones.append({
            'antecedente': antecedente,
            'consecuente': consecuente,
            'soporte_a': soporte_a,
            'confianza': round(conf * 100, 1),
            'lift': round(lift(antecedente, consecuente), 1)
        })

df_asociaciones = pd.DataFrame(asociaciones)
df_asociaciones.sort_values(by='lift', ascending=False, inplace=True)



                            antecedente                 consecuente  \
224                   Cilantro Orgánico                     Limones   
185               Cebolla Roja Orgánica           Cilantro Orgánico   
61                         Ajo Orgánico   Cebolla Amarilla Orgánica   
106  Apio Orgánico en Ramillete Pequeño                  Zanahorias   
102  Apio Orgánico en Ramillete Pequeño             Pepino Orgánico   
..                                  ...                         ...   
274                      Crema Orgánica    Espinacas Baby Orgánicas   
304                Frambuesas Orgánicas                      Fresas   
38                    Aguacate Orgánico  Bolsa de Bananas Orgánicas   
309                Frambuesas Orgánicas                Limón Grande   
15               Aguacate Hass Orgánico                      Banana   

     soporte_a  confianza  lift  
224   0.035389       25.4   3.6  
185   0.034297       12.8   3.6  
61    0.055218       20.1   3.5  
106   0.034

In [115]:
df_asociaciones

,antecedente,consecuente,soporte_a,confianza,lift
224,Cilantro Orgánico,Limones,0.035389,25.4,3.6
185,Cebolla Roja Orgánica,Cilantro Orgánico,0.034297,12.8,3.6
61,Ajo Orgánico,Cebolla Amarilla Orgánica,0.055218,20.1,3.5
106,Apio Orgánico en Ramillete Pequeño,Zanahorias,0.034279,12.0,3.3
102,Apio Orgánico en Ramillete Pequeño,Pepino Orgánico,0.034279,12.7,3.1
...,...,...,...,...,...
274,Crema Orgánica,Espinacas Baby Orgánicas,0.038279,8.7,0.7
304,Frambuesas Orgánicas,Fresas,0.069068,5.3,0.7
38,Aguacate Orgánico,Bolsa de Bananas Orgánicas,0.089293,13.0,0.7
309,Frambuesas Orgánicas,Limón Grande,0.069068,5.8,0.7


In [ ]:
# creamos una tabla con los productos únicos

In [119]:
df_unicos=df[['id_producto','id_departamento','nombre_producto']].drop_duplicates()

In [121]:
df_unicos

,id_producto,id_departamento,nombre_producto
0,49683,4,Pepino Kirby
1,13176,4,Bolsa de Bananas Orgánicas
2,47209,4,Aguacate Hass Orgánico
3,28985,4,Col Rizada Orgánica de Michigan
4,17794,4,Zanahorias
5,21903,4,Espinacas Baby Orgánicas
7,27966,4,Frambuesas Orgánicas
9,24852,4,Banana
10,31717,4,Cilantro Orgánico
11,47766,4,Aguacate Orgánico


In [133]:
# unimos ambas tablas
df_asociaciones_enriquecido=pd.merge(df_asociaciones, df_unicos, left_on='antecedente',right_on='nombre_producto', how='left').drop(columns='nombre_producto', axis=1)


In [137]:
df_asociaciones_enriquecido

,antecedente,consecuente,soporte_a,confianza,lift,id_producto,id_departamento
0,Cilantro Orgánico,Limones,0.035389,25.4,3.6,31717,4
1,Cebolla Roja Orgánica,Cilantro Orgánico,0.034297,12.8,3.6,8518,4
2,Ajo Orgánico,Cebolla Amarilla Orgánica,0.055218,20.1,3.5,24964,4
3,Apio Orgánico en Ramillete Pequeño,Zanahorias,0.034279,12.0,3.3,44359,4
4,Apio Orgánico en Ramillete Pequeño,Pepino Orgánico,0.034279,12.7,3.1,44359,4
...,...,...,...,...,...,...,...
394,Crema Orgánica,Espinacas Baby Orgánicas,0.038279,8.7,0.7,49235,16
395,Frambuesas Orgánicas,Fresas,0.069068,5.3,0.7,27966,4
396,Aguacate Orgánico,Bolsa de Bananas Orgánicas,0.089293,13.0,0.7,47766,4
397,Frambuesas Orgánicas,Limón Grande,0.069068,5.8,0.7,27966,4


In [141]:
# exportamos la tabla
#df_asociaciones_enriquecido.to_csv('nombre', index=False, sep=',',decimal=',')